## Resampling raster data

In [1]:
import geopandas as gpd


import matplotlib.pyplot as plt
from matplotlib import colors

import rasterio
from rasterio.mask import mask

import numpy as np

In [2]:
from rasterio.warp import reproject, Resampling

In [6]:
reprojected_file1 = 'germany_population_reprojected.tif'
reprojected_file2 = 'germany_elevation_reprojected.tif'

In [7]:
with rasterio.open(reprojected_file1) as src1:
    res1 = src1.res
    print('resolution1: ', res1)

resolution1:  (745.7190107123781, 745.7190107123781)


In [8]:
with rasterio.open(reprojected_file2) as src2:
    res2 = src2.res
    print('resolution2: ', res2)

resolution2:  (74.56942202189228, 74.56942202189228)


#### Resolution for the two gids doesn't match -> solving this by making sure the resolutions are the same! 
#### Method: RESAMPLING

In [ ]:
### File1 is the reference for resampling 

with rasterio.open(reprojected_file1) as ref_src:
    target_transform = ref_src.transform
    target_crs = ref_src.crs
    target_width = ref_src.width
    target_height = ref_src.height
    target_resolution = ref_src.res
    target_meta = ref_src.meta.copy()

In [10]:
resmapled_file2 = 'germany_elevation_reprojected_resampled.tif'

In [14]:
with rasterio.open(reprojected_file2) as src:

    aligned_meta = src.meta.copy()
    aligned_meta.update({
        'crs': target_crs,
        'transform': target_transform,
        'width': target_width,
        'height': target_height,
        'dtype': 'float64'
    })


    aligned_data = np.empty((target_height, target_width), dtype=np.float64)

    reproject(
        source = src.read(1),
        destination = aligned_data,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=target_transform,
        dst_crs=target_crs,
        resampling=Resampling.bilinear
        
    )

In [15]:
with rasterio.open(resmapled_file2, 'w', **aligned_meta) as dst:
    dst.write(aligned_data, 1)

In [16]:
import os

In [17]:
size_file1 = os.path.getsize(reprojected_file1)
size_file1

8833350

In [18]:
size_file2 = os.path.getsize(resmapled_file2)
size_file2

8833350